## Configuração do Sistema

In [27]:
import control
from control.matlab import *  # MATLAB-like funcontrolions
from matplotlib import pyplot as plt

%matplotlib

AA = [[0, 1, 0],
      [0, -3.21, 20470],
      [414.93*10**6, 8.72*10**6, 10.39*10**6]]

BB = [[0], [0], [-414.93*10**-6]]

CC = [0, 1, 0]
DD = 0

sistema_ss = ss(AA, BB, CC, 0)

sistema_tf = tf(sistema_ss)
sistema_tf

Using matplotlib backend: TkAgg


TransferFunction(array([-3.72529030e-09, -8.49386597e+00, -7.32421875e-01]), array([ 1.00000000e+00, -1.03899968e+07, -1.78531752e+11, -8.49361710e+12]))

## Resposta ao Degrau

In [28]:
plt.figure(1)
yout, T = step(sistema_ss)
plt.plot(T.T, yout.T)
plt.show()


c:\Users\mathe\OneDrive\Documentos\UNIFACS\controle-3\controle\lib\site-packages\control\timeresp.py:1054: RuntimeWarning: overflow encountered in matmul
  xout[:, i] = (Ad @ xout[:, i-1]
c:\Users\mathe\OneDrive\Documentos\UNIFACS\controle-3\controle\lib\site-packages\control\timeresp.py:1056: RuntimeWarning: invalid value encountered in matmul
  yout = C @ xout + D @ U


### Diagrama de Body

In [29]:
plt.figure(2)
mag, phase, om = bode(sistema_ss, logspace(-2, 2), plot=True)
plt.show()

## Local das Raízes

In [30]:
plt.figure(3)
rlocus(sistema_ss)
plt.show()